# NIH Chest XRay DenseNet

Imports

In [12]:
import os
import pandas as pd
import numpy as np
from glob import glob
from PIL import Image

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models

from sklearn.model_selection import train_test_split

Load dataframe

In [13]:
df = pd.read_csv("archive/Data_Entry_2017.csv")

# target column is “Finding Labels”
df["Finding Labels"] = df["Finding Labels"].astype(str)

# single-label encoding
df["target"] = df["Finding Labels"].apply(lambda x: 0 if x == "No Finding" else 1)

Create map for images

In [14]:
def build_image_folder_map(root="archive"):
    image_map = {}

    folders = glob(os.path.join(root, "images_*", "images"))

    for folder in folders:
        files = os.listdir(folder)
        for f in files:
            if f.lower().endswith((".png", ".jpg", ".jpeg")):
                image_map[f] = folder
    
    return image_map

image_to_folder = build_image_folder_map("archive")
print("Mapped image files:", len(image_to_folder))

Mapped image files: 112120


Fix the dataset

In [15]:
class NIHFolderDataset(Dataset):
    def __init__(self, df, image_map, transform=None):
        self.df = df.reset_index(drop=True)
        self.image_map = image_map
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        filename = row["Image Index"]

        # location of this image
        folder = self.image_map[filename]
        full_path = os.path.join(folder, filename)

        image = Image.open(full_path).convert("RGB")
        label = row["target"]

        if self.transform:
            image = self.transform(image)

        return image, label

Train/validation split

In [16]:
train_df, val_df = train_test_split(df, test_size=0.20, random_state=42)

Transforms

In [17]:
train_tfms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

test_tfms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
])

Create datasets/loaders

In [18]:
train_dataset = NIHFolderDataset(train_df, image_to_folder, transform=train_tfms)
val_dataset   = NIHFolderDataset(val_df,   image_to_folder, transform=test_tfms)

# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=0,
    pin_memory=False
)
val_loader   = DataLoader(val_dataset,   batch_size=32, shuffle=False, num_workers=4)


In [19]:
print(f"Number of batches in train_loader: {len(train_loader)}")
print(f"Number of batches in train_dataset: {len(train_dataset)}")

Number of batches in train_loader: 2803
Number of batches in train_dataset: 89696


Model (DenseNet121)

In [21]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

model = models.densenet121(weights="DEFAULT")
model.classifier = nn.Linear(model.classifier.in_features, 1)
model = model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Load trained model safely
state_dict = torch.load("model_weights.pth", map_location="cpu")
model.load_state_dict(state_dict)
model = model.to(device)

model.eval()

cpu


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

Training loop

In [10]:
epochs = 3

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for i, (images, targets) in enumerate(train_loader):
        # print(f"Processing batch {i}...", flush=True)
        images, targets = images.to(device), targets.float().to(device)

        optimizer.zero_grad()
        outputs = model(images).squeeze(1)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Print batch-level loss every 50 batches
        if i % 50 == 0:
            print(f"Epoch {epoch+1}, Batch {i}, Loss = {loss.item():.4f}")

    # Print average epoch loss
    print(f"Epoch {epoch+1}/{epochs}, Average Loss = {total_loss/len(train_loader):.4f}")

Epoch 1, Batch 0, Loss = 0.6799
Epoch 1, Batch 50, Loss = 0.5462
Epoch 1, Batch 100, Loss = 0.5834
Epoch 1, Batch 150, Loss = 0.6385
Epoch 1, Batch 200, Loss = 0.7162
Epoch 1, Batch 250, Loss = 0.5845
Epoch 1, Batch 300, Loss = 0.7010
Epoch 1, Batch 350, Loss = 0.7053
Epoch 1, Batch 400, Loss = 0.6794
Epoch 1, Batch 450, Loss = 0.6966
Epoch 1, Batch 500, Loss = 0.5644
Epoch 1, Batch 550, Loss = 0.5387
Epoch 1, Batch 600, Loss = 0.5624
Epoch 1, Batch 650, Loss = 0.5089
Epoch 1, Batch 700, Loss = 0.6968
Epoch 1, Batch 750, Loss = 0.6620
Epoch 1, Batch 800, Loss = 0.6010
Epoch 1, Batch 850, Loss = 0.5918
Epoch 1, Batch 900, Loss = 0.5644
Epoch 1, Batch 950, Loss = 0.5715
Epoch 1, Batch 1000, Loss = 0.7259
Epoch 1, Batch 1050, Loss = 0.4834
Epoch 1, Batch 1100, Loss = 0.6307
Epoch 1, Batch 1150, Loss = 0.6201
Epoch 1, Batch 1200, Loss = 0.7288
Epoch 1, Batch 1250, Loss = 0.5533
Epoch 1, Batch 1300, Loss = 0.6937
Epoch 1, Batch 1350, Loss = 0.4974
Epoch 1, Batch 1400, Loss = 0.5602
Epoch 1,

In [11]:
torch.save(model.state_dict(), "model_weights.pth")

torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    'epoch': epoch,
    'loss': loss
}, "checkpoint.pth")

Validation

In [ ]:
model.eval()
val_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.float().to(device)

        outputs = model(inputs)

        # BCEWithLogitsLoss expects shape [batch, 1]
        if labels.dim() == 1:
            labels = labels.unsqueeze(1)

        loss = criterion(outputs, labels)
        val_loss += loss.item()

        # Convert logits → probabilities → predicted classes (0/1)
        preds = torch.sigmoid(outputs)
        preds = (preds > 0.5).int()

        correct += (preds == labels.int()).sum().item()
        total += labels.size(0)

avg_val_loss = val_loss / len(val_loader)
val_accuracy = correct / total

print(f"Validation Loss: {avg_val_loss:.4f}  |  Accuracy: {val_accuracy:.4f}")

In [ ]:
# Generated by ChatGPT

def evaluate_accuracy(model, dataloader, device, task="binary"):
    """
    task = "binary"      -> BCEWithLogitsLoss, output shape [B], labels 0/1
    task = "multiclass"  -> CrossEntropyLoss,  output shape [B, C], labels 0..C-1
    task = "multilabel"  -> BCEWithLogitsLoss, output shape [B, C], labels 0/1 for each class
    """

    model.eval()
    correct = 0
    total = 0

    # For multilabel:
    total_labels = 0
    correct_labels = 0

    with torch.no_grad():
        for images, targets in dataloader:
            images = images.to(device)
            targets = targets.to(device)

            outputs = model(images)

            # ------------------------------
            # BINARY CLASSIFICATION
            # ------------------------------
            if task == "binary":
                probs = torch.sigmoid(outputs).squeeze(1)
                preds = (probs >= 0.5).long()

                correct += (preds == targets).sum().item()
                total += targets.size(0)

            # ------------------------------
            # MULTI-CLASS CLASSIFICATION
            # ------------------------------
            elif task == "multiclass":
                preds = outputs.argmax(dim=1)
                correct += (preds == targets).sum().item()
                total += targets.size(0)

            # ------------------------------
            # MULTI-LABEL CLASSIFICATION
            # ------------------------------
            elif task == "multilabel":
                probs = torch.sigmoid(outputs)
                preds = (probs >= 0.5).long()

                correct_labels += (preds == targets).sum().item()
                total_labels += targets.numel()

            else:
                raise ValueError("task must be 'binary', 'multiclass', or 'multilabel'")

    if task in ("binary", "multiclass"):
        return correct / total
    else:
        return correct_labels / total_labels

In [ ]:
acc = evaluate_accuracy(model, val_loader, device)
print(f"Accuracy: {acc:.4f}")

In [ ]:
hard_examples = []

with torch.no_grad():
    for inputs, labels, paths in val_loader:
        inputs = inputs.to(device)
        labels = labels.float().to(device)
        labels = labels.unsqueeze(1)

        outputs = model(inputs)
        batch_loss = torch.nn.functional.binary_cross_entropy_with_logits(
            outputs, labels, reduction='none'
        )

        # Save per-image loss
        for l, p in zip(batch_loss.cpu().numpy(), paths):
            hard_examples.append((l, p))

roc-auc